### Escolha uma empresa de sua preferência e estime o seu β (beta), coletando dados sobre o preço mensal de suas ações nos últimos 5 anos e do índice que represente o mercado (S&P 500, Ibovespa etc) de acordo com as características da empresa escolhida.

Ação escolhida: **SAPR4 (SANEPAR)**

Índice representante: **IBOVESPA**

In [ ]:
#TODO: comparativo com Ambipar, Casan, Copasa, Sabesp e Orizon (AMBP3, CASN4, CSMG3, SBSP3, ORVR3)

In [1]:
import yfinance as yf
import pandas as pd

start_date = '2019-07-31'
end_date = '2023-08-31'

sapr4 = yf.Ticker("SAPR4.SA")
ibovespa = yf.Ticker("^BVSP")

In [24]:
sapr4_data = sapr4.history(start=start_date, end=end_date)
ibovespa_data = ibovespa.history(start=start_date, end=end_date)

sapr4_monthly_return = sapr4_data['Close'].resample('M').ffill().pct_change()
ibovespa_monthly_return = ibovespa_data['Close'].resample('M').ffill().pct_change()

monthly_returns_df = pd.DataFrame({
    'SAPR4 Monthly Return': sapr4_monthly_return,
    'IBOVESPA Monthly Return': ibovespa_monthly_return
})

monthly_returns_df.drop(monthly_returns_df.index[0], inplace=True)

monthly_returns_df.tail()


,SAPR4 Monthly Return,IBOVESPA Monthly Return
Date,,
2023-04-30 00:00:00-03:00,0.052341,0.025029
2023-05-31 00:00:00-03:00,0.060209,0.037374
2023-06-30 00:00:00-03:00,0.088889,0.090017
2023-07-31 00:00:00-03:00,-0.003912,0.032654
2023-08-31 00:00:00-03:00,0.106888,-0.036148


In [39]:
ibov_mean = monthly_returns_df['IBOVESPA Monthly Return'].mean()

In [49]:
import statsmodels.api as sm

X = monthly_returns_df['IBOVESPA Monthly Return']#.values.reshape(-1, 1)
y = monthly_returns_df['SAPR4 Monthly Return']#.values.reshape(-1, 1)

X = sm.add_constant(X) # adding a constant
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
print_model = model.summary()

print(print_model)

                             OLS Regression Results                             
Dep. Variable:     SAPR4 Monthly Return   R-squared:                       0.431
Model:                              OLS   Adj. R-squared:                  0.419
Method:                   Least Squares   F-statistic:                     35.57
Date:                  Sat, 09 Sep 2023   Prob (F-statistic):           3.05e-07
Time:                          17:39:01   Log-Likelihood:                 70.231
No. Observations:                    49   AIC:                            -136.5
Df Residuals:                        47   BIC:                            -132.7
Df Model:                             1                                         
Covariance Type:              nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const 

O beta estimado para a ação SAPR4 é de 0,68, ou seja, para cada 1% de variação no IBOVESPA, a SAPR4 varia 0,68%.

### Utilizando o β (beta) encontrado no exercício anterior, calcule o custo de capital próprio (expectativa de retorno) dessa empresa utilizando o modelo Capital Asset Pricing Model (CAPM). Lembre-se de utilizar como taxa livre de risco e prêmio de risco alguma das opções que discutimos na disciplina, que faça sentido para a empresa em questão. (Por exemplo: para uma empresa brasileira, encontrar um valor para a Rf do Brasil e prêmio de risco para o mercado brasileiro.)

Opção escolhida como livre de risco: **Tesouro Prefixado 2029 (11,06% a.a)**

Risk Free Rate mensal: **0,87%**

Opção escolhida como livre de risco: **Tesouro Prefixado 2023 (6,85% a.a)**

Risk Free Rate mensal: **0,55%**

In [43]:
Rf = 1.0685**(1/12) - 1
Rf

0.005536586873271299

In [47]:
beta = model.params.values[1]  # Beta SAPR4
Rm = ibov_mean - Rf  # Market Risk Premium

# Calculando o CAPM
capm = Rf + beta * Rm

print(f"Beta: {beta:.4f} Risk Free: {Rf:.4f} Market Risk Premium: {Rm:.4f}\n")

print("CAPM:", capm)

Beta: 0.6896 Risk Free: 0.0055 Market Risk Premium: 0.0003

CAPM: 0.005718965599621765


### Encontre o α de Jensen (alpha de Jensen) para a empresa escolhida. Como você avalia a performance dos gestores dessa empresa? Você compraria ações dessa empresa hoje? Justifique sua resposta.

In [50]:
sapr_mean = monthly_returns_df['SAPR4 Monthly Return'].mean()

In [51]:
# Cálculo do Alpha de Jensen
alpha = sapr_mean - (Rf + beta * (Rm - Rf))

print("O Alpha de Jensen é:", alpha)

O Alpha de Jensen é: 0.003300828913826323


Alpha positivo: Indica que o fundo superou as expectativas em relação ao risco assumido, o que sugere um bom desempenho em relação ao mercado.

### Qual porcentagem de risco dessa empresa pode ser reduzida através de uma carteira de investimentos diversificada?

### Considere 2 cenários: a) aumento dos preços do petróleo; b) uma nova pandemia. Como esses dois eventos poderiam afetar o β (beta) dessa empresa?